The first step in the fitting process is getting the data.
It is assumed that data is always of the type as Neil gave, in a xlsx file with different sheets.
the first row has the names of the different data sets. 
and each further row has the corresponding data points for each set.

Also the relevant parameters needed to pre-process the data are asked here.
Like what data is being cut off to remove divergencies/negative values and removing the.
washing peak, if existant

In [ ]:
#Choice of parameters
#"../231013_DtxR_data_Michael"
filename = "../230920_data_for_Michael" #xlsx file that contains all data in the different sheets
#extract the data to analyse from the xlsx file
sheetNr = 4
#TrpR: [2,10,12]   = [#5, #13, #22]
#CoCl:  [4,6,8,14,16] = [#8,#4,#10,#24,#7]
setsUsedForFitting = 3  #   > 0, <  number of concentrations, typically have to kick out some data due to poor behavior (negative concentrations etc),
                        # in particular the low concentration cases

nrWashings = 2 #define the number of washings used




Here the data is read out of the xlsx file and stored for later usage
one can also see here the names of the data showing e.g. what concentrations were used

In [ ]:
######################################
######################################
## Choice of sheet
######################################
######################################

from python_scripts.xlsx_file_read_extractor import *
from python_scripts.helper_functions import *
from python_scripts.Fit_functions import *
from python_scripts.Rate_equation_solution import *




used_workbook = Get_workbook_from_xlsx_file(filename, True)
usedSheet = used_workbook[used_workbook.sheetnames[sheetNr]]   #sheet decides the experiment, use 2+ for experiments, 2/3, 4/5 etc are the same, just different channels
dataDict = Get_data_from_sheet(usedSheet) #contains all data in form of dictionary, splitting it up by the used concentrations (and also time extra)
allkeys = list(dataDict.keys()) #keys for the data dict
print('name of the used sheet:')
print(usedSheet.title)
print('found key names:')
for x in allkeys:
    print(x)



Generation of ideal test-data

This cell is used to create your own data for testing puproses
If not needed, comment everything out

In [ ]:
Fit_ideal_data = False # don't comment this one out, automizes later stuff


# #Create ideal data for fitting
# import matplotlib.pyplot as plt
# from python_scripts.Rate_equation_solution import *
# t_vals = [0.3 *x for x in range(300)]
# endTimeCutoff = 90 #define the time in s when data should no longer be considered
# startCutOffsets = [20,20]   #define how many data points of the washing peaks should be cut off, before the first washing is registered with my program  i.e. peakposition + offset
# cutEndOffset = 25   
# t0_ideal = [0,50]
# nrWashings = len(t0_ideal)
# weightFactor = 2
# setsUsedForFitting = 2
# lineark23 = True
# Fit_ideal_data = True
# startValDict = dict()
# startValDict['k12'] = [[0.03,0],[True,False]]
# startValDict['k21'] = [[0.2,0.2],[False,False]]
# startValDict['k23'] = [[0.01,0.002],[lineark23,False]]
# startValDict['k32'] = [[0.02,0.02],[False,False]]
# startValDict['C1_start'] = [30, False]
# startValDict['C2_start'] = [0.00001,False]   #not 0 because logarithm used, would potentially give -inf
# startValDict['C3_start'] = [0,False]   #not 0 because logarithm used, would potentially give -inv
# startValDict['t_0'] = [t0_ideal,[True,True]]

# boundValDict = dict()
# boundValDict['k12'] = [0,2]  #lower and upper values [[lower bounds], [upper bounds]], will partially ignore if false chosen for individual parts
# boundValDict['k21'] = [0,2]   #same lower/upper bounds for all params
# boundValDict['k23'] = [0,2]
# boundValDict['k32'] = [0,2]
# boundValDict['C1_start'] = [5,45.5] #if no fit, then no keyword stored of it, see C2,C3
# boundValDict['t_0'] = [[x - 3 for x in startValDict['t_0'][0]],[x + 5 for x in startValDict['t_0'][0]]]


# d1,d2 = Order_fitvalDict(startValDict,boundValDict,nrWashings)
# usedFitFunction = Storage_of_fit_choice(d1,weightFactor)
# motorconcentrations = [0.15625, 0.3125, 0.625, 1.25, 2.5, 5, 10]
# a12Val = 0.07  #slope for k12
# a23Val = 0.0002 #slope for k23, if linear



# concentrations = [] #contains all concentrations for the different motor concentrations
# for x in motorconcentrations:
#     listOfParameters = usedFitFunction.current_variable_params_as_list()
#     listOfParameters[0] = a12Val * x
#     if lineark23:
#         listOfParameters[1] = a23Val * x
#     concentrations.append(usedFitFunction.call_full_function(t_vals,*listOfParameters))
#     plt.plot(t_vals, concentrations[-1][0], label=x)


# dataDict = dict()
# dataDict['Time (s)'] = t_vals
# for x, y in zip(motorconcentrations, concentrations):
#     measuredConcentration = []
#     for i in range(len(y[0])):
#         measuredConcentration.append(y[1][i] + weightFactor * y[2][i])
#     dataDict[x] = measuredConcentration

# allkeys = list(dataDict.keys())
# plt.legend()

In [ ]:
Fit_ideal_data = False # don't comment this one out, automizes later stuff


# #Create ideal data for fitting
# import matplotlib.pyplot as plt
# from python_scripts.Rate_equation_solution import *
# t_vals = [0.3 *x for x in range(300)]
# endTimeCutoff = 90 #define the time in s when data should no longer be considered
# startCutOffsets = [20,20]   #define how many data points of the washing peaks should be cut off, before the first washing is registered with my program  i.e. peakposition + offset
# cutEndOffset = 25   
# t0_ideal = [0,50]
# nrWashings = len(t0_ideal)
# weightFactor = 2
# setsUsedForFitting = 2
# lineark23 = True
# Fit_ideal_data = True
# startValDict = dict()
# startValDict['k12'] = [[0.03,0],[True,False]]
# startValDict['k21'] = [[0.2,0.2],[False,False]]
# startValDict['k13'] = [[0.01,0.00],[lineark23,False]]
# startValDict['k31'] = [[0.02,0.02],[False,False]]
# startValDict['C1_start'] = [30, False]
# startValDict['C2_start'] = [0.00001,False]   #not 0 because logarithm used, would potentially give -inf
# startValDict['C3_start'] = [0,False]   #not 0 because logarithm used, would potentially give -inv
# startValDict['t_0'] = [t0_ideal,[True,True]]

# boundValDict = dict()
# boundValDict['k12'] = [0,2]  #lower and upper values [[lower bounds], [upper bounds]], will partially ignore if false chosen for individual parts
# boundValDict['k21'] = [0,2]   #same lower/upper bounds for all params
# boundValDict['k13'] = [0,2]
# boundValDict['k31'] = [0,2]
# boundValDict['C1_start'] = [5,45.5] #if no fit, then no keyword stored of it, see C2,C3
# boundValDict['t_0'] = [[x - 3 for x in startValDict['t_0'][0]],[x + 5 for x in startValDict['t_0'][0]]]


# d1,d2 = Order_fitvalDict(startValDict,boundValDict,nrWashings)
# usedFitFunction = Storage_of_fit_choice(d1,weightFactor)
# motorconcentrations = [0.15625, 0.3125, 0.625, 1.25, 2.5, 5, 10]
# a12Val = 0.07  #slope for k12
# a23Val = 0.005 #slope for k23, if linear



# concentrations = [] #contains all concentrations for the different motor concentrations
# for x in motorconcentrations:
#     listOfParameters = usedFitFunction.current_variable_params_as_list()
#     listOfParameters[0] = a12Val * x
#     if lineark23:
#         listOfParameters[1] = a23Val * x
#     concentrations.append(usedFitFunction.call_full_function(t_vals,*listOfParameters))
#     plt.plot(t_vals, concentrations[-1][0], label=x)


# dataDict = dict()
# dataDict['Time (s)'] = t_vals
# for x, y in zip(motorconcentrations, concentrations):
#     measuredConcentration = []
#     for i in range(len(y[0])):
#         measuredConcentration.append(y[1][i] + weightFactor * y[2][i])
#     dataDict[x] = measuredConcentration

# allkeys = list(dataDict.keys())
# plt.legend()

Here the used data is plotted, showing what will be fitted

Use the plot to consider what should be cut out from the data
Especially log data is helpful, to look for negative values

In [ ]:
#plot the data to see what it looks like and what has to be done later
#just for visualization, nothing has to be done here
import matplotlib.pyplot as plt




timeKey = allkeys[0]
usedDataKeys = allkeys[setsUsedForFitting:]
if setsUsedForFitting != 1:
    print('Note: not all data is being used')
maxy = 0
timevals = dataDict[timeKey]
yvalslist = []
plt.rcParams.update({'font.size': 12})


for x in usedDataKeys:
    yvals = dataDict[x]
    if maxy < max(yvals):
        maxy = max(yvals)
    yvalslist.append(yvals)    
    # plt.plot(timevals,yvals, label=x)

# plt.figure(figsize=[5,3])
fig, ax_dict = plt.subplot_mosaic([['vis']],figsize=[6,4])#,
                                #   empty_sentinel="BLANK")


for (x,y) in zip(reversed(yvalslist),reversed(usedDataKeys)):
    data = x
    # data = [z + 0.045 for z in x ]
    times = timevals
    # list_shrinker(times,300)
    # list_shrinker(data,300)
    ax_dict['vis'].plot(timevals,  data , 'x-',markersize=1,lw=1,  label=y)
    # ax_dict['log'].plot(timevals, x, 'x-',markersize=1,lw=1,  label=y)

# ax_dict[x].set_title(x + ' fit params')
ax_dict['vis'].set_ylabel('RU')
ax_dict['vis'].set_xlabel(timeKey)
# ax_dict['vis'].set_yscale('log')
# ax_dict['vis'].legend(title='Added motor (nM)')
# ax_dict['log'].set_xlabel(timeKey)
# ax_dict['log'].set_yscale('log')
# ax_dict['log'].set_title('logscale')
# ax_dict['vis'].set_xlim([0,0.7*dataDict[timeKey][-1]])
# ax_dict['vis'].set_xlim([150,210])

# ax_dict['vis'].set_ylim([0,10])
# ax_dict['log'].set_xlim([-1,dataDict[timeKey][-1]])
    
plt.tight_layout()    
# plt.suptitle(f'Data of exp. {usedSheet.title}')

# plt.show()

# for (x,y) in zip(yvalslist,usedDataKeys):
#     plt.plot(timevals, x, 'x-',markersize=1,lw=1,  label=y)
# plt.xlabel(timeKey)
# plt.ylabel('RU')   
# plt.ylim([0,3]) 
# plt.title('Data_exp_' + usedSheet.title + '.pdf' )
# plt.savefig('Full_data_exp_' + usedSheet.title + '.pdf' )
# plt.ylim([0,maxy])
# plt.ylim([1e0,1e2])
plt.xlim([-1,1 + dataDict[timeKey][-1]])
# plt.savefig('Data_' + usedSheet.title + "_dtxRLigands.pdf")

In [ ]:
# timecut = timevals
# list_shrinker(timecut,500)
# for (x,y) in zip(yvalslist,usedDataKeys):
#     dataOfInterest = x
#     list_shrinker(dataOfInterest, 500)
#     plt.plot(timevals, dataOfInterest, 'x-',markersize=1,lw=1,  label=y)
# plt.xlabel(timeKey)
# plt.ylabel('RU')   
# plt.ylim([0,4]) 
# plt.title('Data_exp_' + usedSheet.title + '.pdf' )
# # plt.savefig('Full_data_exp_' + usedSheet.title + '.pdf' )
# # plt.ylim([0,maxy])
# # plt.ylim([1e0,1e2])
# # plt.xlim([-1,dataDict[timeKey][-1]])
# plt.legend()
# plt.savefig('Data_' + usedSheet.title + ".pdf")

HEre the pre-processing is being done. Choose good cut offs to get the best data

In [ ]:
################################################
################################################
#vary parameters

endTimeCutoff = 400 #define the time in s when data should no longer be considered

startCutOffsets = [20,28]   #define how many data points of the washing peaks should be cut off, before the first washing is registered with my program  i.e. peakposition + offset
cutEndOffset = 34         #define how much before a registered peak should be considered
#clean data such, that there are no negative values nor vanishing ones, as we fit the log of the data

################################################
################################################


timeKey = allkeys[0]
usedDataKeys = allkeys[setsUsedForFitting:]
yvalsdictall = dict()
timevalall = []
t0_estimates = []
if not Fit_ideal_data:
    yvalsdictall, timevalall, t0_estimates = cut_peaks_and_shrink_data(dataDict, timeKey, usedDataKeys,nrWashings, endRunTime = endTimeCutoff,cutStartOffset = startCutOffsets,cutEndOffset = cutEndOffset)
else:
    timevalall = dataDict[timeKey]
    t0_estimates = t0_ideal
    for x in usedDataKeys:
        yvalsdictall[x] = dataDict[x]


print('found times of washings')
print(t0_estimates)
for x in usedDataKeys:
    plt.plot(timevalall,yvalsdictall[x], label=x, marker='o',markersize=4)
plt.legend(title='[TW] in nM')    
plt.xlabel(timeKey)
plt.ylabel('RU')
plt.title(f'Data of exp. {usedSheet.title}')
# plt.ylim([-2,30])
# plt.savefig('Full_data_exp_' + usedSheet.title + '.pdf' )
# plt.ylim([0,maxy])
#choose to show everything in log scale
# plt.yscale('log')
# plt.ylim([1e0,1e2])
# plt.xlim([0,dataDict[timeKey][-1]])
# plt.savefig('Data_' + usedSheet.title + ".pdf")

Here the fit region is chosen for the 1. step (i.e. 2. washing area)

One can try to just use the found washing steps from the peaks

In [ ]:
#testcutting
import copy
# chosenTInterval = [t0_estimates[0],dataDict[timeKey][-1]]#,t0_estimates[1]]#,dataDict[timeKey][-1]]
# chosenTInterval = [t0_estimates[0],t0_estimates[1]]#,dataDict[timeKey][-1]]
chosenTInterval = [t0_estimates[1],dataDict[timeKey][-1]]#,
tmpdict = copy.deepcopy(yvalsdictall)
tmpdict[timeKey] = timevalall
data = Cut_timeslot_from_dataDict(tmpdict,chosenTInterval,timeKey)
# timevals = data[timeKey]
yvalsdict = dict()
timevals = data[timeKey]
for x in reversed(usedDataKeys):
    yvalsdict[x] = data[x]
    plt.plot(data[timeKey],data[x],marker='o', label=x)
    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    # print(str(len(data[timeKey])) +'vs. ' + str(len(data[x])))

# plt.yscale('log')
plt.legend(title="motor (nM)")   
plt.xlabel('time in s')
plt.ylabel('RU')

In [ ]:
chosenTInterval

First fitting step, determining k_21, k_32

In [ ]:
weightFactor = 2 #whether C3 has double the weight for fitting
C2maxEstimate = 9
C3maxEstimate = 1

startValDict = dict()
startValDict['k12'] = [[0,0],[False,False]]
startValDict['k21'] = [[0.0, 0.19],[False,True]]
startValDict['k23'] = [[0.0,0],[False,False]]
startValDict['k32'] = [[0.00,0.002],[False,True]]
startValDict['C1_start'] = [0.0, False]
startValDict['C2_start'] = [30.0001,True]   #not 0 because logarithm used, would potentially give -inf
startValDict['C3_start'] = [3,True]   #not 0 because logarithm used, would potentially give -inv
startValDict['t_0'] = [t0_estimates,[False,True]]

boundValDict = dict()
boundValDict['k21'] = [0,2]   #same lower/upper bounds for all params
boundValDict['k23'] = [0,2]
boundValDict['k32'] = [0,2]
boundValDict['t_0'] = [[x - 3 for x in startValDict['t_0'][0]],[x + 5 for x in startValDict['t_0'][0]]]


# Fit_to_function_multiple_washes(timelist,datalist,startValDict,boundValDict,nrWashings)

allResults = dict()
maxconcentration = max(yvalsdict.keys())
for x in reversed(yvalsdict.keys()):
    print('fitting for ' + str(x))
    copydictstartvals = copy.deepcopy(startValDict)
    copydictstartvals['C2_start'][0] =  C2maxEstimate * x /maxconcentration
    copydictstartvals['C3_start'][0] =  C3maxEstimate * x /maxconcentration

    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    resultfit = Fit_to_function_multiple_washes(timevals,yvalsdict[x],copydictstartvals, boundValDict,nrWashings,weightFactor,3000)
    allResults[x] = resultfit
    plt.plot(timevals,yvalsdict[x], label=x)
    # print(resultfit[0])
    resultparams = resultfit[0]
    # resultparams[1] = 5* resultparams[1]
    plotOfFit = resultfit[2].call_full_function(timevals,*resultparams)
    # print(resultfit[2].fit_positions_and_names())
    plt.plot(timevals,np.array(plotOfFit[1]) +  weightFactor * np.array(plotOfFit[2]), '--')
# plt.legend(title="Motor (nM)")
plt.xlabel(timeKey)
plt.ylabel('RU')
plt.yscale('log')
plt.show()


#Calculate k32, k21 and t0 from fits
concentrations = []
# find k21, k32, t_0
fitdataDict = dict()
names = []
for x in allResults.keys():
    concentrations.append(x)
    # print('current case: ' + str(x))
    # print(allResults[x][2].fit_positions_and_names())
    names.append(allResults[x][2].fit_positions_and_names())



for x in names[0]:
   fitdataDict[x[0]] = [[],[]] #value and std of a fit value

for x in allResults.keys():
    stdvec = np.sqrt(np.diag(allResults[x][1]))
    # print(stdvec.shape)
    for m,y,z in zip(fitdataDict.keys(), allResults[x][0], stdvec):
        fitdataDict[m][0].append(y)
        fitdataDict[m][1].append(z) 

print('the fit params are')
print(fitdataDict.keys())
for x in fitdataDict.keys():
    print(fitdataDict[x])

allVals = dict()
allvariances = []

for x in fitdataDict:
    fitdata = fitdataDict[x][0] # the fitted values for a fit choice
    allVals[x] =sum(fitdata)/len(fitdata) #mean value (only useful if constant)
    print(fitdataDict[x][0])
    if 'k21' == x or 'k32' == x:
        plt.plot(concentrations,fitdataDict[x][0], 'x',label=x)
plt.legend()

k21fitstep1 = allVals['k21']
k32fitstep1 = allVals['k32']

# for x in usedDataKeys[3:]:
#     perr = np.sqrt(np.diag(allResults[x][1]))
#     # print(len(allResults[x][0]))
#     # print(perr)

#     allVals.append(allResults[x][0][consideredcase])
#     allvariances.append(np.diag(allResults[x][1])[consideredcase])
#     print(f'{allResults[x][0][consideredcase]:0.5f}' + " +- " + f'{perr[consideredcase]:0.6f}' )

# totalVariance = sum(allvariances)/len(allvariances)
# VarianceFromMEans = 0
# for x in allVals:
#     for y in allVals:
#         VarianceFromMEans += (x-y)**2/len(allvariances)**2

# totalSTD = np.sqrt(totalVariance + VarianceFromMEans)

# print(f'{sum(allVals)/len(allVals):0.4f}' + '+-' + f'{totalSTD:0.4f}' )   







2. step, fitting first wash with fix k21, k32 from before

In [ ]:
#testcutting
import copy
# chosenTInterval = [t0_estimates[0],dataDict[timeKey][-1]]#,t0_estimates[1]]#,dataDict[timeKey][-1]]
chosenTInterval = [t0_estimates[0],t0_estimates[1]]#,dataDict[timeKey][-1]]
# chosenTInterval = [t0_estimates[1],dataDict[timeKey][-1]]#,
tmpdict = copy.deepcopy(yvalsdictall)
tmpdict[timeKey] = timevalall
data = Cut_timeslot_from_dataDict(tmpdict,chosenTInterval,timeKey)
# timevals = data[timeKey]
yvalsdict = dict()
timevals = data[timeKey]
for x in reversed(usedDataKeys):
    yvalsdict[x] = data[x]
    plt.plot(data[timeKey],data[x],marker='o', label=x)
    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    # print(str(len(data[timeKey])) +'vs. ' + str(len(data[x])))


plt.legend(title="motor (nM)")   
plt.xlabel('time in s')
plt.ylabel('RU')

In [ ]:

# startValDict = dict()
# startValDict['k12'] = [[0.2,0],[True,False]]
# startValDict['k21'] = [[k21fitstep1, k21fitstep1],[False,False]]
# startValDict['k23'] = [[0.02,0],[True,False]]
# startValDict['k32'] = [[k32fitstep1,k32fitstep1],[False,False]]
# startValDict['C1_start'] = [11.0, True]
# startValDict['C2_start'] = [0.0001,False]   #not 0 because logarithm used, would potentially give -inf
# startValDict['C3_start'] = [0,False]   #not 0 because logarithm used, would potentially give -inv
# startValDict['t_0'] = [t0_estimates,[True,False]]

# boundValDict = dict()
# boundValDict['k21'] = [0,2]   #same lower/upper bounds for all params
# boundValDict['k23'] = [0,2]
# boundValDict['k32'] = [0,2]
# # boundValDict['C1_start'] = [7,20]
# boundValDict['t_0'] = [[x - 3 for x in startValDict['t_0'][0]],[x + 5 for x in startValDict['t_0'][0]]]


# # Fit_to_function_multiple_washes(timelist,datalist,startValDict,boundValDict,nrWashings)

# allResults = dict()
# maxconcentration = max(yvalsdict.keys())
# for x in reversed(yvalsdict.keys()):
#     print('fitting for ' + str(x))
#     copydictstartvals = copy.deepcopy(startValDict)
#     # copydictstartvals['C2_start'][0] =  C2maxEstimate * x /maxconcentration
#     # copydictstartvals['C3_start'][0] =  C3maxEstimate * x /maxconcentration

#     # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
#     resultfit = Fit_to_function_multiple_washes(timevals,yvalsdict[x],copydictstartvals, boundValDict,nrWashings,weightFactor,3000)
#     allResults[x] = resultfit
#     plt.plot(timevals,yvalsdict[x], label=x)
#     # print(resultfit[0])
#     resultparams = resultfit[0]
#     # resultparams[1] = 5* resultparams[1]
#     plotOfFit = resultfit[2].call_full_function(timevals,*resultparams)
#     # print(resultfit[2].fit_positions_and_names())
#     plt.plot(timevals,np.array(plotOfFit[1]) +  weightFactor * np.array(plotOfFit[2]), '--')
# plt.legend(title="Motor (nM)")
# plt.xlabel(timeKey)
# plt.ylabel('RU')
# plt.show()


# #Calculate k32, k21 and t0 from fits
# concentrations = []
# # find k21, k32, t_0
# fitdataDict = dict()
# names = []
# for x in allResults.keys():
#     concentrations.append(x)
#     # print('current case: ' + str(x))
#     # print(allResults[x][2].fit_positions_and_names())
#     names.append(allResults[x][2].fit_positions_and_names())



# for x in names[0]:
#    fitdataDict[x[0]] = [[],[]] #value and std of a fit value

# for x in allResults.keys():
#     stdvec = np.sqrt(np.diag(allResults[x][1]))
#     # print(stdvec.shape)
#     for m,y,z in zip(fitdataDict.keys(), allResults[x][0], stdvec):
#         fitdataDict[m][0].append(y)
#         fitdataDict[m][1].append(z) 

# print('the fit params are')
# print(fitdataDict.keys())
# for x in fitdataDict.keys():
#     print(fitdataDict[x])

# allVals = dict()
# allvariances = []

# a12fitstep2 = 0
# a23fitstep2 = 0
# C1fitstep2 = 0
# for x in fitdataDict:
#     # print(x)
#     fitdata = fitdataDict[x][0] # the fitted values for a fit choice
#     allVals[x] =sum(fitdata)/len(fitdata) #mean value (only useful if constant)
#     print(x + str(allVals[x]))
#     plt.show()
#     plt.plot(concentrations,fitdata,label=x)
#     fiting,fitcov = curve_fit(lin_fit,concentrations, fitdata)
#     val =lambda x, *params: lin_fit(x,*params)
#     plt.plot(concentrations, val(concentrations, *fiting), label = f'a={fiting[0]:0.5f}')
#     plt.legend()
#     if x == 'k12':
#         a12fitstep2 = fiting[0]
#     elif x == 'k23':
#         a23fitstep2 = fiting[0]
#     elif x == 'C1_start':
#         fiting,fitcov = curve_fit(lambda x,param: lin_fit(x,0,param),concentrations[3:], fitdata[3:])
#         C1fitstep2 = fiting[0]
#         # plt.show()
#         # plt.plot(concentrations, val(concentrations, 0,*fiting), label = f'b={fiting[0]:0.5f}')
#         # plt.plot(concentrations[3:], fitdata[3:], label = f'b={fiting[0]:0.5f}')



# print('value' + str( C1fitstep2))
# # k21fitstep1 = allVals['k21']
# # k32fitstep1 = allVals['k32']





3. Step, full data fit for consistency

In [ ]:
#testcutting
import copy
chosenTInterval = [t0_estimates[0],dataDict[timeKey][-1]]#,t0_estimates[1]]#,dataDict[timeKey][-1]]
# chosenTInterval = [t0_estimates[0],t0_estimates[1]]#,dataDict[timeKey][-1]]
# chosenTInterval = [t0_estimates[1],dataDict[timeKey][-1]]#,
tmpdict = copy.deepcopy(yvalsdictall)
tmpdict[timeKey] = timevalall
data = Cut_timeslot_from_dataDict(tmpdict,chosenTInterval,timeKey)
# timevals = data[timeKey]
yvalsdict = dict()
timevals = data[timeKey]
for x in reversed(usedDataKeys):
    yvalsdict[x] = data[x]
    plt.plot(data[timeKey],data[x],marker='o', label=x)
    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    # print(str(len(data[timeKey])) +'vs. ' + str(len(data[x])))


plt.legend(title="motor (nM)")   
plt.xlabel('time in s')
plt.ylabel('RU')

In [ ]:

startValDict = dict()
startValDict['k12'] = [[0.2,0],[True,False]]
startValDict['k21'] = [[k21fitstep1, k21fitstep1],[False,False]]
startValDict['k23'] = [[0.01,0],[True,False]]
startValDict['k32'] = [[k32fitstep1,k32fitstep1],[False,False]]
startValDict['C1_start'] = [10, True]
startValDict['C2_start'] = [0.0001,False]   #not 0 because logarithm used, would potentially give -inf
startValDict['C3_start'] = [0,False]   #not 0 because logarithm used, would potentially give -inv
startValDict['t_0'] = [t0_estimates,[True,True]]

boundValDict = dict()
boundValDict['k21'] = [0,2]   #same lower/upper bounds for all params
boundValDict['k23'] = [0,2]
boundValDict['k32'] = [0,2]
boundValDict['C1_start'] = [2,50]
boundValDict['t_0'] = [[x - 0.003 for x in startValDict['t_0'][0]],[x + 1 for x in startValDict['t_0'][0]]]

a12fitstep2 = 0.028
# Fit_to_function_multiple_washes(timelist,datalist,startValDict,boundValDict,nrWashings)
a23fitstep2 = 0.0002
allResults = dict()
maxconcentration = max(yvalsdict.keys())
for x in reversed(yvalsdict.keys()):
    print('fitting for ' + str(x))
    copydictstartvals = copy.deepcopy(startValDict)
    copydictstartvals['k12'][0][0] =  a12fitstep2 * x 
    copydictstartvals['k23'][0][0] =  a23fitstep2 * x 


    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    resultfit = Fit_to_function_multiple_washes(timevals,yvalsdict[x],copydictstartvals, boundValDict,nrWashings,weightFactor,3000)
    allResults[x] = resultfit
    plt.plot(timevals,yvalsdict[x], label=x)
    # print(resultfit[0])
    resultparams = resultfit[0]
    # resultparams[1] = 5* resultparams[1]
    plotOfFit = resultfit[2].call_full_function(timevals,*resultparams)
    # print(resultfit[2].fit_positions_and_names())
    plt.plot(timevals,np.array(plotOfFit[1]) +  weightFactor * np.array(plotOfFit[2]), '--')
plt.legend(title="Motor (nM)")
plt.xlabel(timeKey)
plt.ylabel('RU')
plt.show()


#Calculate k32, k21 and t0 from fits
concentrations = []
# find k21, k32, t_0
fitdataDict = dict()
names = []
for x in allResults.keys():
    concentrations.append(x)
    # print('current case: ' + str(x))
    # print(allResults[x][2].fit_positions_and_names())
    names.append(allResults[x][2].fit_positions_and_names())



for x in names[0]:
   fitdataDict[x[0]] = [[],[]] #value and std of a fit value

for x in allResults.keys():
    stdvec = np.sqrt(np.diag(allResults[x][1]))
    # print(stdvec.shape)
    for m,y,z in zip(fitdataDict.keys(), allResults[x][0], stdvec):
        fitdataDict[m][0].append(y)
        fitdataDict[m][1].append(z) 

print('the fit params are')
print(fitdataDict.keys())
for x in fitdataDict.keys():
    print(fitdataDict[x])

allVals = dict()
allvariances = []

for x in fitdataDict:
    # print(x)
    fitdata = fitdataDict[x][0] # the fitted values for a fit choice
    allVals[x] =sum(fitdata)/len(fitdata) #mean value (only useful if constant)
    print(x + str(allVals[x]))
    plt.show()
    plt.plot(concentrations,fitdata,'x',label=x)
    plt.xlabel('concentration in nM')
    # fitfct = lambda x,param : lin_fit(x,param,0)
    # fitfct = lambda x,param : lin_fit(x,0,param)
    fitfct = lambda x,param : lin_fit(x,param,0)
    fiting,fitcov = curve_fit(fitfct,concentrations, fitdata)
    val =lambda x, *params: fitfct(x,*params)
    # if x == 'k12':
    #     a12fitstep2 = fiting[0]
    # elif x == 'k23':
    #     a23fitstep2 = fiting[0]
    # elif x == 'C1_start':
    #     C1fitstep2 = allVals[x]    
    if x == 'k12' or x == 'k23':
        plt.plot(concentrations, val(concentrations, *fiting), label = f'a={fiting[0]:0.4f}')

    plt.legend()



End of fitting protocol 

Here manual fitting can be done:

In [ ]:
#testcutting
import copy
# chosenTInterval = [t0_estimates[0],dataDict[timeKey][-1]]#,t0_estimates[1]]#,dataDict[timeKey][-1]]
# chosenTInterval = [t0_estimates[0],t0_estimates[1]]#,dataDict[timeKey][-1]]
chosenTInterval = [t0_estimates[1],dataDict[timeKey][-1]]#,
tmpdict = copy.deepcopy(yvalsdictall)
tmpdict[timeKey] = timevalall
data = Cut_timeslot_from_dataDict(tmpdict,chosenTInterval,timeKey)
# timevals = data[timeKey]
yvalsdict = dict()
timevals = data[timeKey]
for x in reversed(usedDataKeys):
    yvalsdict[x] = data[x]
    plt.plot(data[timeKey],data[x],marker='o', label=x)
    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    # print(str(len(data[timeKey])) +'vs. ' + str(len(data[x])))


plt.legend(title="motor (nM)")   
plt.xlabel('time in s')
plt.ylabel('RU')

Here the first fitting is done

In [ ]:
################################################
################################################
#if fitting does not work, might have to adjust fit params here
#visualization might also later change, if other fit params are used
#Last difficult thing to do is the generation of arbitrary figures
#might have to try to generalize this via a function for all data of interest
################################################
################################################

startValDict = dict()
startValDict['k12'] = [[0,0],[True,False]]
startValDict['k21'] = [[0.21, 0.21],[False,False]]
startValDict['k23'] = [[0.0,0],[True,True]]
startValDict['k32'] = [[0.0023,0.0023],[False,False]]
startValDict['C1_start'] = [14.0, True]
startValDict['C2_start'] = [0.0001,False]   #not 0 because logarithm used, would potentially give -inf
startValDict['C3_start'] = [0,False]   #not 0 because logarithm used, would potentially give -inv
# startValDict['t_0'] = [[0.12,29.7],[False,False]]
startValDict['t_0'] = [t0_estimates,[False,True]]

boundValDict = dict()
boundValDict['k12'] = [0,6]  #lower and upper values [[lower bounds], [upper bounds]], will partially ignore if false chosen for individual parts
boundValDict['k21'] = [0,2]   #same lower/upper bounds for all params
boundValDict['k23'] = [0,2]
boundValDict['k32'] = [0,2]
boundValDict['C1_start'] = [5,45.5] #if no fit, then no keyword stored of it, see C2,C3
boundValDict['t_0'] = [[x - 3 for x in startValDict['t_0'][0]],[x + 5 for x in startValDict['t_0'][0]]]


# Fit_to_function_multiple_washes(timelist,datalist,startValDict,boundValDict,nrWashings)

allResults = dict()
C2maxEstimate = 8
C3maxEstimate = 1
maxconcentration = max(yvalsdict.keys())
a23Val = 2e-4
a12Val = 0.0248
for x in reversed(yvalsdict.keys()):
    print('fitting for ' + str(x))
    copydictstartvals = copy.deepcopy(startValDict)
    copydictstartvals['k12'][0][0] =  a12Val * x
    copydictstartvals['k23'][0][0] =  a23Val * x
    # copydictstartvals['C2_start'][0] =  C2maxEstimate * x /maxconcentration
    # copydictstartvals['C3_start'][0] =  C3maxEstimate * x /maxconcentration

    weightFactor = 2
    # print(str(len(timevals)) +'vs. ' + str(len(yvalsdict[x])))
    resultfit = Fit_to_function_multiple_washes(timevals,yvalsdict[x],copydictstartvals, boundValDict,nrWashings,weightFactor,3000)
    allResults[x] = resultfit
    plt.plot(timevals,yvalsdict[x], label=x)
    # print(resultfit[0])
    resultparams = resultfit[0]
    # resultparams[1] = 5* resultparams[1]
    plotOfFit = resultfit[2].call_full_function(timevals,*resultparams)
    # print(resultfit[2].fit_positions_and_names())
    plt.plot(timevals,np.array(plotOfFit[1]) +  weightFactor * np.array(plotOfFit[2]), '--')
plt.legend(title="Motor (nM)")
plt.xlabel(timeKey)
plt.ylabel('RU')
# plt.yscale('log')
# plt.savefig('fitMotorCocl_singleC3.pdf')

In [ ]:
allVals = []
allvariances = []

consideredcase = 2

for x in usedDataKeys[3:]:
    perr = np.sqrt(np.diag(allResults[x][1]))
    # print(len(allResults[x][0]))
    # print(perr)

    allVals.append(allResults[x][0][consideredcase])
    allvariances.append(np.diag(allResults[x][1])[consideredcase])
    print(f'{allResults[x][0][consideredcase]:0.5f}' + " +- " + f'{perr[consideredcase]:0.6f}' )

totalVariance = sum(allvariances)/len(allvariances)
VarianceFromMEans = 0
for x in allVals:
    for y in allVals:
        VarianceFromMEans += (x-y)**2/len(allvariances)**2

totalSTD = np.sqrt(totalVariance + VarianceFromMEans)

print(f'{sum(allVals)/len(allVals):0.4f}' + '+-' + f'{totalSTD:0.4f}' )   

Here the fit parameters are stored and the concentrations shown at the end of the second washing

In [ ]:

#here the rest of plotting all the fitting parameters, for understanding what happened
#also the interesting vals, then finished with my part for fitting all
#probably best to create another class/function that extracts whatever i want from the dict of interest
interestingstuffCs = [[],[],[]]
xvals = []
names = []
for x in allResults.keys():
    xvals.append(x)
    print('current case: ' + str(x))
    print(allResults[x][2].fit_positions_and_names())
    names.append(allResults[x][2].fit_positions_and_names())
    t0_fitted = [allResults[x][0][-1]]
    print(allResults[x][2].current_variable_params_as_list())
    t0Concentrations = allResults[x][2].call_full_function(t0_fitted,*allResults[x][0])
    interestingstuffCs[0].append(t0Concentrations[0][0])
    interestingstuffCs[1].append(t0Concentrations[1][0])
    interestingstuffCs[2].append(t0Concentrations[2][0])

plt.plot(xvals, interestingstuffCs[0], 'X--', label='C1')
plt.plot(xvals, interestingstuffCs[1], 'X--', label='C2')
plt.plot(xvals, interestingstuffCs[2], 'X--', label='C3')
plt.xlabel('Motor concentration in nM')
plt.ylabel('Concentrations')
# plt.yscale('log')
plt.legend()
# plt.savefig("ConcentrationsfigCoclMotor_secondW.pdf")



Here one can plot the different fit parameters in dep. of [TW] to see what the fit found

In [ ]:
#here the rest of plotting all the fitting parameters, for understanding what happened
#also the interesting vals, then finished with my part for fitting all
#probably best to create another class/function that extracts whatever i want from the dict of interest
interestingstuff = [[],[],[],[],[],[],[],[],[]]
namelist = []
for x in names[0]:
    namelist.append(x[0] + 'w'+ str(x[1] + 1))

xvals = []
for x in allResults.keys():
    xvals.append(x)
    t0_fitted = [allResults[x][0][-1]]
    k_washings = allResults[x][0]
    print(allResults[x][0])
    interestingstuff[0].append(k_washings[0])
    interestingstuff[1].append(k_washings[1])
    interestingstuff[2].append(k_washings[2])
    # interestingstuff[3].append(k_washings[3])
    # interestingstuff[4].append(k_washings[4])
    # interestingstuff[5].append(k_washings[5])
    # interestingstuff[6].append(k_washings[6])
    # interestingstuff[7].append(k_washings[7])
    # interestingstuff[8].append(k_washings[8])


# plt.plot(xvals, interestingstuff[0], 'X--', label='k12w1')
# # plt.plot(xvals, interestingstuff[1], 'X--', label='k21w1')
# fitk12w1,_ = curve_fit(lin_fit,xvals[:3], interestingstuff[1][:3])

# # # print(fitk12w1)
# plt.plot(xvals[:3], lin_fit(xvals[:3],*fitk12w1), label=f"slope={fitk12w1[0]:.5f}/s/nM")
# plt.plot(xvals, interestingstuff[1], 'X--', label='k21w1')
# plt.plot(xvals, interestingstuff[2], 'X--', label='k21w2')

# fitk23w1,_ = curve_fit(lin_fit,xvals[:3], interestingstuff[0][:3])
# plt.plot(xvals[:3], lin_fit(xvals[:3],*fitk23w1), label=f"a={fitk23w1[0]:.5f}/s/nM, b={fitk23w1[1]:.4f}1/s")
# # 

fitchoiceposi = 1
# fitk23w1,fitcov = curve_fit(lambda x,param: lin_fit(x,param,0),xvals[:], interestingstuff[fitchoiceposi][:])
# val = lambda x,param: lin_fit(x,param,0)
fitk23w1,fitcov = curve_fit(lin_fit,xvals[:], interestingstuff[fitchoiceposi][:])
val =lambda x, *params: lin_fit(x,*params)

print(val(xvals[:],*fitk23w1))
plt.plot(xvals[:], val(xvals[:],*fitk23w1), label=f"a={fitk23w1[0]:.5f}/s/nM")

print(f'a={fitk23w1[0]:0.5f} +- {np.sqrt(np.diag(fitcov))[0]:}')
# print(f'b={fitk23w1[1]:0.5f} +- {np.sqrt(np.diag(fitcov))[1]:}')

plt.plot(xvals, interestingstuff[fitchoiceposi], 'X--', label=namelist[fitchoiceposi])


# plt.plot(xvals, interestingstuff[0], 'X--', label=namelist[0])
# plt.plot(xvals, interestingstuff[1], 'X--', label=namelist[1])
# plt.plot(xvals, interestingstuff[2], 'X--', label=namelist[2])
# plt.plot(xvals, interestingstuff[3],  'X--', label=namelist[3])
# plt.plot(xvals, interestingstuff[4], 'X--', label=namelist[4])
# plt.plot(xvals, interestingstuff[5], 'X--', label=namelist[5])
# plt.plot(xvals, interestingstuff[6], 'X--', label=namelist[6])
# plt.plot(xvals, interestingstuff[7], 'X--', label=namelist[7])
# plt.plot(xvals, interestingstuff[8], 'X--', label=namelist[8])

# plt.plot(xvals, interestingstuff[3], 'X--', label='k23w1')
# for x,y in zip(interestingstuff, namelist):
#     plt.plot(xvals, x, 'X--', label=y)

# plt.plot(xvals, interestingstuff[5], 'X--', label='k32w1')
# plt.plot(xvals, interestingstuff[6], 'X--', label='k32w2')


# print('mean k32w2' + str(sum(interestingstuff[4])/len(interestingstuff[4])))
# print('k21 val: ')
# meanK21 = sum(interestingstuff[2])/len(interestingstuff[2])
# print(meanK21)

# plt.xlabel('Motor concentration in nM')
# plt.ylabel('Rate in 1/s')
# # plt.axhline(y=0.0037,  label='k_32',ls='--')
# # plt.yscale('log')
plt.legend()
# # plt.savvefig('ratefitW2k23_CoclMot.pdf')
plt.show()
plt.plot(xvals, interestingstuff[0], 'X--', label='C1')
plt.xlabel('Motor concentration in nM')
plt.ylabel('C1')
# # plt.savefig("freeDNAconcentration_motorCoCl.pdf")
# # 

Here the 2. washing step

In [ ]:

def lin_fit(vals, a,b):
    return a*np.array(vals) + b

In [ ]:
#compact plot of all important things
fig, ax_dict = plt.subplot_mosaic([['fullFit','rates'],['C1','Csat2Wash']],figsize=[9,4])#,
                                #   empty_sentinel="BLANK")


# for (x,y) in zip(yvalslist,usedDataKeys):
#     ax_dict['vis'].plot(timevals, x, 'x-',markersize=1,lw=1,  label=y)
#     ax_dict['log'].plot(timevals, x, 'x-',markersize=1,lw=1,  label=y)



for x in yvalsdict.keys():
    ax_dict['fullFit'].plot(timevals,yvalsdict[x], label=x)
    # print(resultfit[0])
    plotOfFit = allResults[x][2].call_full_function(timevals,*allResults[x][0])
    # print(resultfit[2].fit_positions_and_names())
    ax_dict['fullFit'].plot(timevals,np.array(plotOfFit[1]) + np.array(plotOfFit[2]), '--')
ax_dict['fullFit'].legend(loc='center')
ax_dict['fullFit'].set_xlabel(timeKey)
ax_dict['fullFit'].set_ylabel('RU')


ax_dict['rates'].plot(xvals, interestingstuff[0], 'X--', label='k12w1')
fitk12w1,_ = curve_fit(lin_fit,xvals, interestingstuff[0])
# print(fitk12w1)
ax_dict['rates'].plot(xvals, lin_fit(xvals,*fitk12w1), label=f"a:{fitk12w1[0]:.2f}/s/nM")
ax_dict['rates'].plot(xvals, interestingstuff[1], 'X--', label='k21w1')
ax_dict['rates'].plot(xvals, interestingstuff[2], 'X--', label='k21w2')
# # plt.plot(xvals, interestingstuff[3], 'X--', label='k23w1')
# # plt.plot(xvals, interestingstuff[4], 'X--', label='k23w2')
# # plt.plot(xvals, interestingstuff[5], 'X--', label='k32w1')
# # plt.plot(xvals, interestingstuff[6], 'X--', label='k32w2')

ax_dict['rates'].legend()
ax_dict['rates'].set_xlabel('concentration in nM')
ax_dict['rates'].set_ylabel('rates')



# plt.xlabel('Motor concentration in nM')
# plt.ylabel('Rates')
# # plt.yscale('log')
# plt.legend()
# plt.show()
ax_dict['C1'].plot(xvals, interestingstuff[7], 'X--', label='C1')
ax_dict['C1'].legend()
ax_dict['C1'].set_xlabel('concentration in nM')
ax_dict['C1'].set_ylabel('Free DNA C1')

# plt.xlabel('Motor concentration in nM')
# plt.ylabel('C1')

plt.plot(xvals, interestingstuffCs[0], 'X--', label='C1')
plt.plot(xvals, interestingstuffCs[1], 'X--', label='C2')
ax_dict['Csat2Wash'].legend()
ax_dict['Csat2Wash'].set_xlabel('concentration in nM')
ax_dict['Csat2Wash'].set_ylabel("C's at 2. washing")
# plt.plot(xvals, interestingstuffCs[2], 'X--', label='C3')

fig.tight_layout()
# plt.savefig('fullFig_fitchoice2.pdf')

In [ ]:
0.2/0.07